<a href="https://colab.research.google.com/github/mostafa-ja/cloud-computing/blob/main/FatTree.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [16]:
import numpy as np

In [68]:
#import k from user's input
k = 8

#open the output file
f = open("output.txt","w")


#Calculating the pods, servers, core switches, aggregation switches

pods_n = int(k)
AggrSwitch_in_pod = int(k/2)
AggrSwitch_n = int(k*AggrSwitch_in_pod)
EdgeSwitch_in_pod = int(k/2)
EdgeSwitch_n = int(k*EdgeSwitch_in_pod)
Servers_in_pod = int(EdgeSwitch_in_pod*(k/2))
Servers_n = int(Servers_in_pod*k)
CoreSwitch_n = int((k/2)**2)

# we address switches and servers with numbers from bottom to top
Servers_range_start = 0  #include zero
Servers_range_end = (Servers_range_start + Servers_n) #exclude the end number
EdgeSwitch_range_start = Servers_range_end
EdgeSwitch_range_end = (EdgeSwitch_range_start + EdgeSwitch_n)
AggrSwitch_range_start = EdgeSwitch_range_end
AggrSwitch_range_end = (EdgeSwitch_range_end + AggrSwitch_n)
CoreSwitch_range_start = AggrSwitch_range_end
CoreSwitch_range_end = (AggrSwitch_range_end + CoreSwitch_n)

f.write(f'Servers_range = [{Servers_range_start},{Servers_range_end}) \n')
f.write(f'EdgeSwitch_range = [{EdgeSwitch_range_start},{EdgeSwitch_range_end}) \n')
f.write(f'AggrSwitch_range = [{AggrSwitch_range_start},{AggrSwitch_range_end}) \n')
f.write(f'CoreSwitch_range = [{CoreSwitch_range_start},{CoreSwitch_range_end}) \n')

30

In [52]:
f.write('Servers_range_start = '+str(Servers_range_start)+ "\n")
f.write('Servers_range_end = '+str(Servers_range_end)+ "\n")
f.write('EdgeSwitch_range_start = '+str(EdgeSwitch_range_start)+ "\n")
f.write('EdgeSwitch_range_end = '+str(EdgeSwitch_range_end)+ "\n")
f.write('AggrSwitch_range_start = '+str(AggrSwitch_range_start)+ "\n")
f.write('AggrSwitch_range_end = '+str(AggrSwitch_range_end)+ "\n")
f.write('CoreSwitch_range_start = '+str(CoreSwitch_range_start)+ "\n")
f.write('CoreSwitch_range_end = '+str(CoreSwitch_range_end)+ "\n")
print('Servers_range_end = ',Servers_range_end)
print('EdgeSwitch_range_end = ',EdgeSwitch_range_end)
print('AggrSwitch_range_end = ',AggrSwitch_range_end)
print('CoreSwitch_range_end = ',CoreSwitch_range_end)
print()

Servers_range_end =  128
EdgeSwitch_range_end =  160
AggrSwitch_range_end =  192
CoreSwitch_range_end =  208



In [69]:
# program goes stage by stage bottom up, first we create links between edge and servers
server_number = 0
for edge in range(EdgeSwitch_range_start,EdgeSwitch_range_end):
  for i in range(int(k/2)):   #each edgeServer connects to (k/2) servers
    out = (f'{edge} \t {server_number} \t 1 \n')
    f.write(out)
    out = (f'{server_number} \t {edge} \t 1 \n')
    f.write(out)
    #f.write(str(edge)+'\t'+str(server_number)+'\t'+"1"+'\n')
    #f.write(str(server_number)+'\t'+str(edge)+'\t'+"1"+'\n')
    server_number += 1



In [ ]:
# counter is way that we address number of edge switches
# counter starts at number of servers because 0 should be included in server counts
counter = Servers_Count

# server counter is a way to address the server for mapping between edge switches and servers
Server_counter = 0

# program goes stage by stage bottom up, first we create links between edge and servers

for i in range(pods_Count):

    for j in range(EdgeSwitch_Count_In_Pods):

        for l in range (k//2):

            out = str(counter)+"\t"+str(Server_counter)+"\t"+"1"+"\n"
            f.write(out)
            

            out = str(Server_counter) + "\t" + str(counter) + "\t" + "1" + "\n"
            f.write(out)
            connected.append((Server_counter,counter ))
            

            Server_counter+=1

        counter += 1

#Handling connection between Aggregation switches and edge switches

In [ ]:
for i in range(pods_Count):
  for j in range(AggrSwitch_Count_In_Pods):
    for l in range(k/2):
      
